In [ ]:
# Reproducible setup
from __future__ import annotations

import sys
from pathlib import Path

import tensorflow as tf

# Make src/ importable regardless of where the notebook is launched
CWD = Path.cwd().resolve()
ROOT = CWD if (CWD / "src").exists() else CWD.parent
sys.path.insert(0, str(ROOT / "src"))

from ff_pipeline.hdf5_utils import H5Spec, validate_h5
from ff_pipeline.training import TrainConfig, train

DATA_DIR = ROOT / "data"
MODELS_DIR = ROOT / "models"

TRAIN_H5 = DATA_DIR / "train.h5"
VAL_H5 = DATA_DIR / "val.h5"
TEST_H5 = DATA_DIR / "test.h5"

spec = H5Spec(image_size=224)
validate_h5(TRAIN_H5, spec)
validate_h5(VAL_H5, spec)
validate_h5(TEST_H5, spec)

print("Data OK:")
print("-", TRAIN_H5)
print("-", VAL_H5)
print("-", TEST_H5)


In [ ]:
# Dataset creation is provided by the reusable module
from ff_pipeline.datasets import make_hdf5_dataset



In [ ]:
# Load datasets
cfg = TrainConfig(
    image_size=224,
    batch_size=16,
    epochs=10,
    learning_rate=1e-4,
    max_train_samples=5000,
    max_val_samples=5000,
    seed=42,
    imagenet_weights=True,
)

train_ds = make_hdf5_dataset(TRAIN_H5, spec=spec, batch_size=cfg.batch_size, shuffle=True, max_samples=cfg.max_train_samples)
val_ds = make_hdf5_dataset(VAL_H5, spec=spec, batch_size=cfg.batch_size, shuffle=False, max_samples=cfg.max_val_samples)


In [ ]:
# Train using the shared pipeline function 
model = train(train_h5=TRAIN_H5, val_h5=VAL_H5, out_dir=MODELS_DIR, cfg=cfg)
model.summary()


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_pad           │ (None, 230, 230,  │          0 │ input_layer_14[0… │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 112, 112,  │      9,472 │ conv1_pad[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 112, 112,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 112, 112,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pad           │ (None, 114, 114,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1_pool          │ (None, 56, 56,    │          0 │ pool1_pad[0][0]   │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 56, 56,    │      4,160 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 56, 56,    │        256 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 56, 56,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 56, 56,    │     36,928 │ conv2_block1_1_r… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_bn   │ (None, 56, 56,    │        256 │ conv2_block1_2_c… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_relu │ (None, 56, 56,    │          0 │ conv2_block1_2_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_conv │ (None, 56, 56,    │     16,640 │ pool1_pool[0][0]  │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_conv │ (None, 56, 56,    │     16,640 │ conv2_block1_2_r… │
│ (Conv2D)            │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_0_c… │
│ (BatchNormalizatio… │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_3_bn   │ (None, 56, 56,    │      1,024 │ conv2_block1_3_c

 Total params: 26,215,305 (100.00 MB)

 Trainable params: 2,627,593 (10.02 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [ ]:
# Optional: inspect layers
print("num_layers:", len(model.layers))
print("first_10:", [l.name for l in model.layers[:10]])


input_layer_14
conv1_pad
conv1_conv
conv1_bn
conv1_relu
pool1_pad
pool1_pool
conv2_block1_1_conv
conv2_block1_1_bn
conv2_block1_1_relu
conv2_block1_2_conv
conv2_block1_2_bn
conv2_block1_2_relu
conv2_block1_0_conv
conv2_block1_3_conv
conv2_block1_0_bn
conv2_block1_3_bn
conv2_block1_add
conv2_block1_out
conv2_block2_1_conv
conv2_block2_1_bn
conv2_block2_1_relu
conv2_block2_2_conv
conv2_block2_2_bn
conv2_block2_2_relu
conv2_block2_3_conv
conv2_block2_3_bn
conv2_block2_add
conv2_block2_out
conv2_block3_1_conv
conv2_block3_1_bn
conv2_block3_1_relu
conv2_block3_2_conv
conv2_block3_2_bn
conv2_block3_2_relu
conv2_block3_3_conv
conv2_block3_3_bn
conv2_block3_add
conv2_block3_out
conv3_block1_1_conv
conv3_block1_1_bn
conv3_block1_1_relu
conv3_block1_2_conv
conv3_block1_2_bn
conv3_block1_2_relu
conv3_block1_0_conv
conv3_block1_3_conv
conv3_block1_0_bn
conv3_block1_3_bn
conv3_block1_add
conv3_block1_out
conv3_block2_1_conv
conv3_block2_1_bn
conv3_block2_1_relu
conv3_block2_2_conv
conv3_block2_2_bn

In [ ]:
# Training is executed in the train() call above.
# Optional: load best checkpoint explicitly.
best_path = MODELS_DIR / "best_model.keras"
model = tf.keras.models.load_model(best_path)
print("Loaded:", best_path)


Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 514ms/step - gender_output_accuracy: 0.5372 - gender_output_loss: 0.7025 - loss: 2.6917 - race_output_accuracy: 0.1804 - race_output_loss: 1.9892
Epoch 1: val_race_output_accuracy improved from -inf to 0.14375, saving model to ../models/best_model.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 39s 647ms/step - gender_output_accuracy: 0.5367 - gender_output_loss: 0.7026 - loss: 2.6913 - race_output_accuracy: 0.1803 - race_output_loss: 1.9887 - val_gender_output_accuracy: 0.5000 - val_gender_output_loss: 0.7196 - val_loss: 2.6470 - val_race_output_accuracy: 0.1437 - val_race_output_loss: 1.9274
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 553ms/step - gender_output_accuracy: 0.5102 - gender_output_loss: 0.6972 - loss: 2.6586 - race_output_accuracy: 0.1548 - race_output_loss: 1.9614
Epoch 2: val_race_output_accuracy improved from 0.14375 to 0.21250, saving model to ../models/best_model.keras
50/50 ━━━━━━━━━━━━━━━━━━━━ 34s 674ms/step - gender_output_accuracy: 0.5097 

/Users/araj/.pyenv/versions/3.11.8/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)



Epoch 7: val_race_output_accuracy did not improve from 0.23750
50/50 ━━━━━━━━━━━━━━━━━━━━ 17s 326ms/step - gender_output_accuracy: 0.5367 - gender_output_loss: 0.6589 - loss: 2.6838 - race_output_accuracy: 0.1327 - race_output_loss: 1.8832 - val_gender_output_accuracy: 0.5250 - val_gender_output_loss: 0.6929 - val_loss: 2.6310 - val_race_output_accuracy: 0.1375 - val_race_output_loss: 1.9382
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 701ms/step - gender_output_accuracy: 0.4692 - gender_output_loss: 0.7030 - loss: 2.6420 - race_output_accuracy: 0.1801 - race_output_loss: 1.9390
Epoch 8: val_race_output_accuracy did not improve from 0.23750
50/50 ━━━━━━━━━━━━━━━━━━━━ 45s 847ms/step - gender_output_accuracy: 0.4702 - gender_output_loss: 0.7029 - loss: 2.6419 - race_output_accuracy: 0.1801 - race_output_loss: 1.9391 - val_gender_output_accuracy: 0.5063 - val_gender_output_loss: 0.7213 - val_loss: 2.6573 - val_race_output_accuracy: 0.1562 - val_race_output_loss: 1.9360
Epoch 9/10
50/50 ━━━━━